In [1]:
import numpy.random as rdm
import numpy as np
import matplotlib.pyplot as plt
import scipy.fftpack

In [12]:
N = 16
config = rdm.normal(size = N)
print(config)
a = rdm.randint(N, size=N)
config_ = config[a]
for i in range(N):
    print(i, a[i], config[i], config_[i])

[-0.08555741  1.50725632  0.13690646  0.71092869  1.58754484  0.11740764
  0.39865512  0.55403766  0.31242014 -1.40851327  0.56541161 -0.13770488
 -0.43812523  0.00906494 -0.92348248 -0.77102371]
0 9 -0.08555740841907816 -1.408513268479845
1 6 1.507256318003856 0.39865511590621744
2 12 0.1369064554274976 -0.43812522789753267
3 3 0.7109286903068494 0.7109286903068494
4 13 1.5875448352667094 0.009064939019026023
5 14 0.11740764028103239 -0.9234824763632197
6 13 0.39865511590621744 0.009064939019026023
7 7 0.5540376611321798 0.5540376611321798
8 0 0.3124201379297892 -0.08555740841907816
9 3 -1.408513268479845 0.7109286903068494
10 3 0.5654116090221523 0.7109286903068494
11 9 -0.13770487669586776 -1.408513268479845
12 12 -0.43812522789753267 -0.43812522789753267
13 15 0.009064939019026023 -0.7710237077970107
14 3 -0.9234824763632197 0.7109286903068494
15 6 -0.7710237077970107 0.39865511590621744


In [17]:
print(config_.sum())
a = 15
print(config_[a])
print(config_[:a].sum() + config_[a+1:].sum())
print(config_.sum() - config_[a])  
print(len(config_))  


-1.2601480531239997
0.39865511590621744
-1.6588031690302167
-1.6588031690302172
16


In [30]:
config = np.arange(5)
print(config)
print(np.outer(config, config))
outer = np.outer(config, config)
outer = (outer.sum(axis=0) - outer.diagonal())
print(outer)
print(outer.sum())

[0 1 2 3 4]
[[ 0  0  0  0  0]
 [ 0  1  2  3  4]
 [ 0  2  4  6  8]
 [ 0  3  6  9 12]
 [ 0  4  8 12 16]]
[ 0  9 16 21 24]
70


In [38]:
N = 8
for i in range(N):
    for j in range(i,N):
        if i != j:
            print(j-i, end=' ')
    print()

1 2 3 4 5 6 7 
1 2 3 4 5 6 
1 2 3 4 5 
1 2 3 4 
1 2 3 
1 2 
1 



In [39]:
h = np.random.normal(size=8)
config = 2*np.random.randint(2, size=8) - 1
print(h * config)

[-0.66538264 -0.90900821  1.14254374 -0.23962994  1.41735136  1.38578713
 -0.8632493   0.14955149]
